In [1]:
from sklearn.model_selection import train_test_split as split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math, random
%matplotlib inline

In [2]:
train = pd.read_csv('student-mat.csv')
train_y = train[['romantic']]
train = train[['sex', 'age', 'famsize', 'traveltime', 'studytime', 'activities', 'goout', 'famrel']]
train_y['romantic'][train_y['romantic']=='no'] = 0
train_y['romantic'][train_y['romantic']=='yes'] = 1
train['sex'][train['sex']=='F'] = 0
train['sex'][train['sex']=='M'] = 1
train['famsize'][train['famsize']=='LE3'] = 0
train['famsize'][train['famsize']=='GT3'] = 1
train['activities'][train['activities']=='no'] = 0
train['activities'][train['activities']=='yes'] = 1
train = train.apply(pd.to_numeric)
print(train.info())
train.describe()

/home/wermarter/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/wermarter/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/wermarter/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 8 columns):
sex           395 non-null int64
age           395 non-null int64
famsize       395 non-null int64
traveltime    395 non-null int64
studytime     395 non-null int64
activities    395 non-null int64
goout         395 non-null int64
famrel        395 non-null int64
dtypes: int64(8)
memory usage: 24.8 KB
None


,sex,age,famsize,traveltime,studytime,activities,goout,famrel
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,0.473418,16.696203,0.711392,1.448101,2.035443,0.508861,3.108861,3.944304
std,0.499926,1.276043,0.453690,0.697505,0.839240,0.500555,1.113278,0.896659
min,0.000000,15.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000
25%,0.000000,16.000000,0.000000,1.000000,1.000000,0.000000,2.000000,4.000000
50%,0.000000,17.000000,1.000000,1.000000,2.000000,1.000000,3.000000,4.000000
75%,1.000000,18.000000,1.000000,2.000000,2.000000,1.000000,4.000000,5.000000
max,1.000000,22.000000,1.000000,4.000000,4.000000,1.000000,5.000000,5.000000


In [3]:
def train_dev_test(X, Y, dist):
    s, m = sum(dist), len(X)
    a, b, c = (np.array(dist) / s)
    train_x, tmp_x, train_y, tmp_y = split(X, Y, test_size=(b+c)/a)
    dev_x, test_x, dev_y, test_y = split(tmp_x, tmp_y, test_size=c/(b+c))
    return train_x, dev_x, test_x, train_y, dev_y, test_y

train_y = train_y.apply(pd.to_numeric).values
train_x, dev_x, test_x, train_y, dev_y, test_y =train_dev_test(train.values, train_y, [7, 1, 2])

In [ ]:
class NeuralNetworkClassifier:
    # Layers start from 0 (input data)
    # Delta, Theta, Error is list() of numpy.array()
    # 
    #
    def __init__(self, hidden_layers=np.array([2]), learning_rate=0.005, reg_lambda=1, interval=300, epsilon=0.0001):
        np.random.seed(37) # For consistency
        self.L = len(hidden_layers)+2 # Number of layers
        self.hidden_layers = hidden_layers
        self.learning_rate = learning_rate
        self.reg_lambda = reg_lambda # Lambda for Regularization
        self.interval = interval # Interval for showing error rate (the norm function)
        self.count = 0 # Supporting the intervals
        self.epsilon = epsilon
        

    def score(self, X, Y, theta=None):
        # Accuracy Scoring
        res = 0
        for x, y in zip(X, Y):
            if self.predict(x, theta) == y:
                res += 1
        return res/len(X)
            
    def predict_proba(self, X, theta=None):
        if theta == None:
            theta = self.trained
        return float(self.FeedForward(X, theta)[0][-1][0])
    
    def predict(self, X, theta=None):
        res = self.predict_proba(X, theta)
        return 1 if res > 0.5 else 0
    
    def fit(self, X, y): # Main
        m, n = X.shape
        layers = np.insert(np.insert(self.hidden_layers, 0, n), self.L-1, 1) # Numbers of nodes
        print(layers)
        theta = list() # Weights of each Layer
        for l in range(self.L-1):
            theta.append(np.random.random((layers[l+1], layers[l]))) # Exclude Bias
        self.trained = self.GradientDescent(X, y, m, theta)
        return "We did it, dude!"
    
    def Loss(self, X, Y, theta=None):
        # Logistic Loss Function
        if theta == None:
            theta = self.trained
        res, m = 0, X.shape[0]
        for x, y  in zip(X, Y):
            predicted = self.predict_proba(x, theta)
            res += self.Cost(predicted, y)
        return (-1/m)*res + self.MassiveRegularizationSummation(theta, m)
    
    def Cost(self, H, y):
        return (1-y)*np.log(1-H)+y*np.log(H)
    
    def GradientDescent(self, X, y, m, theta):
        derivative = self.TakeDerivatives(X, y, m, theta)
        dev = float("inf")
        while self.norm(derivative, "L2-norm") > self.epsilon:
            #theta = theta - self.learning_rate*derivative # Exclude Bias
            for (i, value), deriv in zip(enumerate(theta), derivative):
                theta[i] = theta[i]*(1-self.learning_rate*self.reg_lambda/m)-self.learning_rate*deriv
            self.trained = theta
            tmp = self.Loss(dev_x, dev_y, theta)
            if tmp > dev:
                print("Development error increased. Early stopping...")
                return theta
            else:
                dev = tmp
                if self.count == 1:
                    print("Dev: " +str(dev))
            derivative = self.TakeDerivatives(X, y, m, theta)
        return theta
    
    def TakeDerivatives(self, X, y, m, theta):
        delta, cost = "None", 0
        for i in range(m):
            a, z = self.FeedForward(X[i], theta)
            cost += self.Cost(a[-1][0], y[i])
            # Element-wise Addition: delta + result of BackPropagation
            tmp = self.BackPropagation(a, z, y[i], theta)
            if delta == "None":
                delta = tmp
            else:
                for i, x in enumerate(tmp):
                    delta[i] += x
        for i, d in enumerate(delta):
            delta[i] *= 1/m
        self.verbal((-1/m)*cost+self.MassiveRegularizationSummation(theta, m))
        return delta
    
    def FeedForward(self, X, theta):
        a, z = list(), list()
        a.append(X)
        z.append(np.NaN)
        for l in range(1, self.L):
            z.append(theta[l-1].dot(a[-1]))
            a.append(self.ActivationFunction(z[-1]))
        return a, z
    
    def BackPropagation(self, a, z, y, theta):
        delta, error = list(), list()
        error.append(a[-1] - y)
        for l in reversed(range(self.L-1)):
            delta.append(error[-1].reshape(-1, 1).dot(a[l].reshape(1, -1)))
            error.append(theta[l].T.dot(error[-1])*self.ActivationFunction(z[l], deriv=True))
        return delta[::-1]

    def verbal(self, x):
        self.count %= self.interval
        if self.count == 0:
            print(x)
        self.count += 1
    
    def norm(self, V, mode="L2-norm"):
        if mode=="L2-norm":
            for i, v in enumerate(V):
                V[i] = (v**2).sum()
            res = np.sqrt((np.array(V)**2).sum())
        if mode=="L1-norm":
            for i, v in enumerate(V):
                V[i] = np.abs(v).sum()
            res = np.array(V).sum()
        return res
    
    def ActivationFunction(self, X, deriv=False):
        # Sigmoid Fucntion
        value = 1/(1+math.e**-X)
        if deriv:
            #Return first derivative of function
            return value*(1-value)
        return value
    def MassiveRegularizationSummation(self, theta, m):
        res = 0 # Calculate the regularization term for cost funtion
        for a in theta:
            res += (a**2).sum()
        return (self.reg_lambda/(2*m))*res
        

In [ ]:
NN = NeuralNetworkClassifier([2, 2], reg_lambda=0.1, learning_rate=0.001)
NN.fit(train_x, train_y)

[8 2 2 1]
[ 0.71920559]
Dev: [ 0.74437718]
[ 0.71379255]
Dev: [ 0.73498425]
[ 0.70904448]
Dev: [ 0.72659021]
[ 0.70485755]
Dev: [ 0.71905132]
[ 0.70114714]
Dev: [ 0.71224906]
[ 0.69784385]
Dev: [ 0.70608506]
[ 0.69489034]
Dev: [ 0.70047709]
[ 0.69223895]
Dev: [ 0.69535602]
[ 0.68984983]
Dev: [ 0.69066334]
[ 0.68768943]
Dev: [ 0.68634931]
[ 0.6857294]
Dev: [ 0.68237137]
[ 0.68394563]
Dev: [ 0.67869293]
[ 0.68231752]
Dev: [ 0.67528241]
[ 0.68082739]
Dev: [ 0.67211239]
[ 0.67946]
Dev: [ 0.66915901]
[ 0.67820216]
Dev: [ 0.66640135]
[ 0.67704239]
Dev: [ 0.66382109]
[ 0.6759707]
Dev: [ 0.66140205]
[ 0.67497832]
Dev: [ 0.65912993]
[ 0.67405757]
Dev: [ 0.65699204]
[ 0.67320166]
Dev: [ 0.65497708]
[ 0.67240459]
Dev: [ 0.65307497]
[ 0.67166105]
Dev: [ 0.65127668]
[ 0.67096632]
Dev: [ 0.6495741]
[ 0.67031616]
Dev: [ 0.64795993]
[ 0.66970683]
Dev: [ 0.64642758]
[ 0.66913494]
Dev: [ 0.6449711]
[ 0.66859745]
Dev: [ 0.64358509]
[ 0.66809164]
Dev: [ 0.64226464]
[ 0.66761505]
Dev: [ 0.64100529]
[ 0.667

In [ ]:
sex, age, famsize, traveltime, studytime, activities, goout, famrel = 1, 17, 1, 1, 4, 0, 0, 5
input_data = np.array([sex, age, famsize, traveltime, studytime, activities, goout, famrel])

In [ ]:
NN.predict_proba(input_data)*100

In [ ]:
NN.score(test_x, test_y)

In [ ]:
NN.score(train_x, train_y)

In [ ]:
NN.trained